In [30]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

In [211]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [278]:
label = np.load("Cas.npy")
label = label.reshape(4989,1)
label = label - 1


# import matplotlib.pyplot as plt
# train_img.shape
# img = Image.fromarray(train_img[2], "RGB")
# img.show()

index = np.where(label==4)
label = np.delete(label, np.where(label == 4))

index_0 = np.where(label==0)
index_0 = index_0[0][:3200]
label = np.delete(label, index_0)
# label = label.reshape(4956-3200, 1)

index_1 = np.where(label==1)
index_1 = index_1[0][:300]
label = np.delete(label, index_1)
label = label.reshape(4956-3200-300, 1)

In [279]:
for i in range(4):
    print(f"The number of images with label {i} is {np.where(label ==i)[0].shape}")

The number of images with label 0 is (380,)
The number of images with label 1 is (365,)
The number of images with label 2 is (357,)
The number of images with label 3 is (354,)


In [280]:
RESIZED_ROOT = os.path.join(os.path.dirname(os.path.abspath('')),'extraction','files_resized_choose')
path_list = os.listdir(RESIZED_ROOT)
path_list.sort(key = lambda x:int(x.split('.')[0]))

file = []
for file_name in path_list:
    file.append(file_name)

file = np.array(file)
file = np.delete(file, index)
file = np.delete(file, index_0)
file = np.delete(file, index_1)
file = file.reshape(4956-3200-300,1)

annotation = np.concatenate((file,label),axis=1)
annotation.shape



(1456, 2)

In [281]:
from sklearn.model_selection import train_test_split
X = file
y = label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [282]:
X_train, y_train
annotation_train = np.concatenate((X_train,y_train),axis=1)
annotation_test = np.concatenate((X_test,y_test),axis=1)
(annotation_train.shape, annotation_test.shape)

((975, 2), (481, 2))

In [283]:
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.DataFrame(annotations_file)
        self.img_dir = img_dir
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        label = int(label)
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [284]:
# AllSet = CustomImageDataset(annotation,"files_resized_choose")
# AllSet.img_labels
# image, label = AllSet.__getitem__(1)

In [285]:
transform = transforms.Compose(
    [transforms.Resize(960), transforms.Grayscale()])

In [286]:
training_data = CustomImageDataset(annotation_train,"files_resized_choose",transform=transform)
test_data = CustomImageDataset(annotation_test,"files_resized_choose",transform=transform)
train_dataloader = DataLoader(training_data, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=4, shuffle=True)

In [288]:
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features.size()}")
# print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0][1].squeeze()
# label = train_labels[0]
# plt.imshow(img)
# plt.show()
# print(f"Label: {label}")


In [293]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 7, stride=3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 7, stride=3)
        self.conv3 = nn.Conv2d(16, 24, 5)
        # self.conv4 = nn.Conv2d(24, 32, 5)
        # self.conv5 = nn.Conv2d(32, 48, 3)
        self.fc1 = nn.Linear(2400, 120)
        self.fc2 = nn.Linear(120, 4)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # x = self.pool(F.relu(self.conv4(x)))
        # x = self.pool(F.relu(self.conv5(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(7, 7), stride=(3, 3))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(7, 7), stride=(3, 3))
  (conv3): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=2400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=4, bias=True)
)


In [294]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [295]:
len(train_dataloader)

244

In [296]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        inputs = inputs.to(torch.float)
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

    # print(f'epoch:{epoch+1} loss:{running_loss / len(train_dataloader)}')

print('Finished Training')

[1,   100] loss: 1.495
[1,   200] loss: 1.388
[2,   100] loss: 1.383
[2,   200] loss: 1.383
Finished Training


In [297]:
correct = 0
total = 0
with torch.no_grad():
    for data in train_dataloader:
        images = data[0].to(device)
        labels = data[1].to(device)
        images = images.to(torch.float)
        output = net(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the train images:{100 * correct / total}%')
print('Number of train images:',total)

Accuracy of the network on the train images:28.307692307692307%
Number of train images: 975


In [298]:
classes =('0','1','2','3')
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# again no gradients needed
with torch.no_grad():
    for data in test_dataloader:
        images = data[0].to(device)
        labels = data[1].to(device)
        images = images.to(torch.float)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: 0     is 98.4 %
Accuracy for class: 1     is 3.2 %
Accuracy for class: 2     is 0.0 %
Accuracy for class: 3     is 1.6 %


In [299]:
PATH = './classification.pth'
torch.save(net.state_dict(), PATH)